# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
R. Keenan  ->  R. Keenan  |  ['R. Keenan']


P. Eitner  ->  P. Eitner  |  ['P. Eitner']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
R. Hoppe  ->  R. Hoppe  |  ['R. Hoppe']
N. Storm  ->  N. Storm  |  ['N. Storm']
M. Flock  ->  M. Flock  |  ['M. Flock']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 118 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.22789


extracting tarball to tmp_2509.22789... done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2509.22789/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'catalog-table' from 'tmp_2509.22789/catalog-table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'summary-table' from 'tmp_2509.22789/summary-table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packa

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


...

Found 110 bibliographic references in tmp_2509.24594/main.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2509.25018


extracting tarball to tmp_2509.25018...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.25021


extracting tarball to tmp_2509.25021...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.22789-b31b1b.svg)](https://arxiv.org/abs/2509.22789) | **Masses, Star-Formation Efficiencies, and Dynamical Evolution of 18,000 HII Regions**  |
|| D. Pathak, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *Accepted for publication in ApJL; main text: 13 pages, 4 figures, 1 table*|
|**Abstract**|            We present measurements of the masses associated with $\sim18,000$ HII regions across 19 nearby star-forming galaxies by combining data from JWST, HST, MUSE, ALMA, VLA, and MeerKAT from the multi-wavelength PHANGS survey. We report 10 pc-scale measurements of the mass of young stars, ionized gas, and older disk stars coincident with each HII region, as well as the initial and current mass of molecular gas, atomic gas, and swept-up shell material, estimated from lower resolution data. We find that the mass of older stars dominates over young stars at $\gtrsim10\rm\,pc$ scales, and ionized gas exceeds the stellar mass in most optically bright HII regions. Combining our mass measurements for a statistically large sample of HII regions, we derive 10 pc scale star-formation efficiencies $\approx6{-}17\%$ for individual HII regions. Comparing each region's self-gravity with the ambient ISM pressure and total pressure from pre-supernova stellar feedback, we show that most optically bright HII regions are over-pressured relative to their own self-gravity and the ambient ISM pressure, and that they are hence likely expanding into their surroundings. Larger HII regions in galaxy centers approach dynamical equilibrium. The self-gravity of regions is expected to dominate over pre-supernova stellar feedback pressure at $\gtrsim130\rm\,pc$ and $60\rm\,pc$ scales in galaxy disks and centers, respectively, but is always sub-dominant to the ambient ISM pressure on HII region scales. Our measurements have direct implications for the dynamical evolution of star-forming regions and the efficiency of stellar feedback in ionizing and clearing cold gas.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.24211-b31b1b.svg)](https://arxiv.org/abs/2509.24211) | **Far-infrared lines hidden in archival deep multi-wavelength surveys: Limits on [CII]-158$μ$m at $z \sim 0.3-2.9$**  |
|| S. Agrawal, J. Aguirre, <mark>R. Keenan</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *Submitted to Astronomy & Astrophysics on July 19, 2025*|
|**Abstract**|            Singly-ionized carbon is theorized to be the brightest emission line feature in star-forming galaxies, and hence an excellent tracer of the evolution of cosmic star formation. Archival maps from far-infrared and sub-millimeter surveys potentially contain the redshifted [CII]-158$\mu$m, hidden in the much brighter continuum emission. We present a search for aggregate [CII]-158$\mu$m line emission across the predicted peak of star formation history by tomographically stacking a high-completeness galaxy catalog on broadband deep maps of the COSMOS field and constraining residual excess emission after subtracting the continuum spectral energy distribution (SED). We obtain constraints on the sky-averaged [CII]-158$\mu$m signal from the three Herschel/SPIRE maps: $11.8\pm10.2$, $11.0\pm8.7$, $9.6\pm9.8$, and $9.2\pm6.6$ $k$Jy/sr at redshifts $z\sim 0.65$, $\sim1.3$, $\sim2.1$, and $\sim2.6$ respectively, corresponding to $1-1.4\sigma$ significance in each bin. Our $3\sigma$ upper limits are in tension with past $z\sim2.6$ results from cross-correlating SDSS-BOSS quasars with high-frequency Planck maps, and indicate a much less dramatic evolution ($\sim\times7.5$) of mean [CII] intensity across the peak of star formation history than collisional excitation models or frameworks calibrated to the tentative PlanckxBOSS measurement. We discuss this tension, particularly in the context of in-development surveys (TIM, EXCLAIM) that will map this [CII] at high redshift resolution. Having demonstrated stacking in broadband deep surveys as a complementary methodology to next-generation spectrometers for line intensity mapping, our novel methods can be extended to upcoming galaxy surveys such as Euclid, as well as to place upper limits on fainter atomic and molecular lines.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.24555-b31b1b.svg)](https://arxiv.org/abs/2509.24555) | **M3DIS - A grid of 3D radiation-hydrodynamics stellar atmosphere models for stellar surveys. II. Carbon-enhanced metal-poor stars**  |
|| <mark>P. Eitner</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>R. Hoppe</mark>, <mark>N. Storm</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *16 pages, 13 figures, accepted by A&A*|
|**Abstract**|            Understanding the origin and evolution of carbon-enhanced metal-poor (CEMP) stars is key to tracing the Galaxy's early chemical enrichment. We investigate how realistic 3D radiation-hydrodynamics (RHD) model atmospheres affect carbon abundances in CEMP stars and implications for their classification and Galactic chemical evolution (GCE). We focus on biases from traditional 1D hydrostatic models. We use the M3DIS code to compute 3D RHD model atmospheres for main-sequence and sub-giant stars over a wide range of metallicities and carbon enhancements. Synthetic spectra of the CH G-band are calculated with 3D radiative transfer and compared to spectra from classical 1D MARCS models. We derive abundance corrections and apply them to a large SAGA database sample to quantify effects on the carbon abundance distribution and CEMP classification. Our new 3D CEMP models predict cooler upper atmospheric layers than in 1D models, resulting in stronger CH absorption and lower inferred carbon abundances by up to -0.9 dex at the lowest metallicities. Carbon enhancement in the atmosphere itself increases molecular opacities and leads to radiative re-heating, partly offsetting adiabatic cooling in 3D models and reducing 3D-1D abundance corrections. Applying these corrections lowers the CEMP fraction by up to 20% below [Fe/H]=-3 and alters the relative contribution of CEMP sub-classes. The fraction of CEMP-no stars increases while the number of CEMP-r/s stars decreases, due to the downward revision of absolute carbon abundances. These changes bring the Galactic carbon distribution into better agreement with GCE models assuming a 20% contribution from faint supernovae. Realistic model atmospheres are essential to reliably reconstruct the Galaxy's early chemical enrichment history.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.24594-b31b1b.svg)](https://arxiv.org/abs/2509.24594) | **Polarization at millimeter wavelengths caused by drifting grains in protoplanetary disks**  |
|| M. Lietzow-Sinjen, S. Reissl, <mark>M. Flock</mark>, S. Wolf |
|*Appeared on*| *2025-09-30*|
|*Comments*| *18 pages, 14 figures, to be published in Astronomy & Astrophysics (A&A)*|
|**Abstract**|            During the evolution of protoplanetary disks, dust grains start to grow, form larger particles, settle to the midplane, and rearrange the disk, mainly by the inward radial drift. Because of this, dust pebbles with an irregular shape usually align mechanically and thus cause polarization signatures in their thermal radiation due to dichroic emission or absorption. The goal of this paper is to evaluate the potential to trace the impact of mechanical grain alignment in protoplanetary disks on the observed degree and orientation of linear polarization at millimeter wavelengths. We combined 3D radiation hydrodynamical simulations to determine the density distribution and the velocity field of gas and dust particles, Monte Carlo dust-gas interaction simulations to calculate the mechanical alignment of dust in a gas flow, and, finally, 3D Monte Carlo polarized radiative transfer simulations to obtain synthetic polarimetric observations. We find that large grains, which contribute the most to the net polarization, are potentially mechanically aligned in the protoplanetary disk under the effect of the vertical shear instability (VSI). Thereby, the drift velocity is parallel to the rotational disk axis. Assuming oblate dust grains that are aligned with their short axis parallel to the direction of the drift velocity, the resulting polarization is usually along the major axis of the disk. This is in contrast to typical drift models that propose either a radial or azimuthal drift velocity component. If hydrodynamical instabilities, such as the VSI, dominate the kinematics in protoplanetary disks, the mechanical alignment of dust is a promising mechanism for grain alignment in these systems. In that case, the resulting millimeter polarization allows us to trace the orientation of aligned millimeter-sized grains.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.23343-b31b1b.svg)](https://arxiv.org/abs/2509.23343) | **Comological Prediction from the joint observation of MeerKAT and CSST at $z$ = 0.4 $\sim$ 1.2**  |
|| Y.-E. Jiang, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *18 pages, 13 figures, and 1 table*|
|**Abstract**|            Cross-correlating neutral hydrogen (HI) 21cm intensity mapping with galaxy surveys provides an effective probe of astrophysical and cosmological information. This work presents a cross-correlation analysis between MeerKAT single-dish HI intensity mapping and Chinese Space Station Survey Telescope (CSST) spectroscopic galaxy surveys in $z=0.4\sim1.2$, which will share a survey area of several thousand square degrees. Utilizing Jiutian-1G cosmological simulation, we simulate the observational data of MeerKAT and CSST with survey areas from $\sim1600$ to $600$ deg$^2$ at $z=0.5$, 0.7, and 1. The effects of beam pattern, polarization leakage, and different foregrounds in the MeerKAT HI intensity mapping are considered in the simulation. After employing foreground removal with the principal component analysis (PCA) method and performing signal compensation, we derive the cross-power spectra of MeerKAT and CSST. We perform the joint constraint using the CSST galaxy auto-power spectra and MeerKAT-CSST cross-power spectra with the least-squares fitting method. The constraint results show that, in the simulated survey area, the relative accuracy can achieve $6\%\sim 8\%$ for the parameter products $\Omega_{\rm HI}b_{\rm HI}b_{g}r_{\rm HI,g}$ and $\Omega_{\rm HI}b_{\rm HI}r_{\rm HI,g}$ at the three redshifts, which is $3\sim4$ times smaller than the current result. These findings indicate that the full MeerKAT-CSST joint observation with thousands of square degrees overlapping survey area can be a powerful probe of cosmic         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.23901-b31b1b.svg)](https://arxiv.org/abs/2509.23901) | **Interpreting deep learning-based stellar mass estimation via causal analysis and mutual information decomposition**  |
|| W. Zhang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *Accepted at Astronomy & Astrophysics; 23 + 12 pages; 8 + 16 figures*|
|**Abstract**|            End-to-end deep learning models fed with multi-band galaxy images are powerful data-driven tools used to estimate galaxy physical properties in the absence of spectroscopy. However, due to a lack of interpretability and the associational nature of such models, it is difficult to understand how the information additional to integrated photometry (e.g., morphology) contributes to the estimation task. Improving our understanding in this field would enable further advances into unraveling the physical connections among galaxy properties and optimizing data exploitation. Therefore, our work is aimed at interpreting the deep learning-based estimation of stellar mass via two interpretability techniques: causal analysis and mutual information decomposition. The former reveals the causal paths between multiple variables beyond nondirectional statistical associations, while the latter quantifies the multicomponent contributions (i.e., redundant, unique, and synergistic) of different input data to the stellar mass estimation. Using data from the Sloan Digital Sky Survey (SDSS) and the Wide-field Infrared Survey Explorer (WISE), we obtained meaningful results that provide physical interpretations for image-based models. Our work demonstrates the gains from combining deep learning with interpretability techniques, and holds promise in promoting more data-driven astrophysical research (e.g., astrophysical parameter estimations and investigations on complex multivariate physical processes).         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25018-b31b1b.svg)](https://arxiv.org/abs/2509.25018) | **CCAT: Mod-Cam Cryogenic Performance and its Impact on 280 GHz KID Array Noise**  |
|| L. T. Lin, et al. -- incl., <mark>Y. Wang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *7 pages, 8 figures, Low temperature detector 2025 proceeding submitted to IEEE Transactions on Applied Superconductivity*|
|**Abstract**|            The CCAT Observatory's Fred Young Submillimeter Telescope (FYST) is designed to observe submillimeter astronomical signals with high precision, using receivers fielding state-of-the-art kinetic inductance detector (KID) arrays. Mod-Cam, a first-light instrument for FYST, serves as a testbed for instrument module characterization, including detailed evaluation of thermal behavior under operating conditions prior to deploying modules in the larger Prime-Cam instrument. Prime-Cam is a first generation multi-band, wide-field camera for FYST, designed to field up to seven instrument modules and provide unprecedented sensitivity across a broad frequency range. We present results from two key laboratory characterizations: an "optically open" cooldown to validate the overall thermal performance of the cryostat, and a "cold load" cooldown to measure the effect of focal plane temperature stability on detector noise. During the optically open test, we achieved stable base temperatures of 1.5 K on the 1 K stage and 85 mK at the detector stage. In the cold load configuration, we measured a detector focal plane RMS temperature stability of 3.2e-5 K. From this stability measurement, we demonstrate that the equivalent power from focal plane thermal fluctuations is only 0.0040% of a 5pW incident photon power for aluminum detectors and 0.0023% for titanium-nitride detectors, a negligible level for CCAT science goals. This highlights the success of the cryogenic system design and thermal management.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25021-b31b1b.svg)](https://arxiv.org/abs/2509.25021) | **CCAT: Mod-Cam Readout Overview and Flexible Stripline Performance**  |
|| B. Keller, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-30*|
|*Comments*| *6 pages, 7 figures, submitted to LTD 2025 conference proceedings*|
|**Abstract**|            The CCAT Observatory's first-light and commissioning instrument, Mod-Cam, is nearing readiness for deployment to the Fred Young Submillimeter Telescope (FYST) in the Atacama Desert in northern Chile. In-lab testing of Mod-Cam and the first CCAT instrument module, a 280 GHz broadband camera fielding over 10,000 kinetic inductance detectors (KIDs), is currently underway. CCAT's first-generation science instrument, Prime-Cam, will field approximately 100,000 KIDs across seven instrument modules. Like Mod-Cam, it employs 46 cm long low-thermal-conductivity flexible circuits ("stripline") between 4 K and 300 K to connect large arrays of multiplexed detectors in each instrument module to readout electronics. The 280 GHz camera currently installed in Mod-Cam uses six striplines to read out its over 10,000 detectors across 18 RF chains, each individual stripline containing six traces. In-lab testing thus far has allowed us to begin optimizing the attenuation in the readout chains for each of the three detector arrays in the 280 GHz module and demonstrate simultaneous readout of all networks using CCAT's RFSoC warm readout electronics. Here we present an overview of the Mod-Cam cold readout and discuss the stripline performance by investigating how its thermal conductivity impacts Mod-Cam and Prime-Cam cryogenic performance. We also report on stripline electrical transmission and crosstalk, identifying transition printed circuit boards (PCBs) as the dominant source of crosstalk. Using this result we outline design improvements to these PCBs that enhance isolation between readout networks.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.22789/./mass_violin_plots.png', 'tmp_2509.22789/./Mnew_MHII_MOld_MMol_vsRcirc.png', 'tmp_2509.22789/./M_vs_R_LHacorr.png', 'tmp_2509.22789/./SFE_est.png', 'tmp_2509.22789/./SFE_MMol_supplement2.png', 'tmp_2509.22789/./SFE_MMol_supplement.png']
copying  tmp_2509.22789/./mass_violin_plots.png to _build/html/
copying  tmp_2509.22789/./Mnew_MHII_MOld_MMol_vsRcirc.png to _build/html/
copying  tmp_2509.22789/./M_vs_R_LHacorr.png to _build/html/
copying  tmp_2509.22789/./SFE_est.png to _build/html/
copying  tmp_2509.22789/./SFE_MMol_supplement2.png to _build/html/
copying  tmp_2509.22789/./SFE_MMol_supplement.png to _build/html/
exported in  _build/html/2509.22789.md
    + _build/html/tmp_2509.22789/./mass_violin_plots.png
    + _build/html/tmp_2509.22789/./Mnew_MHII_MOld_MMol_vsRcirc.png
    + _build/html/tmp_2509.22789/./M_vs_R_LHacorr.png
    + _build/html/tmp_2509.22789/./SFE_est.png
    + _build/html/tmp_2509.22789/./SFE_MMol_supplement2.png
    + _build/html/tmp

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Masses, Star-Formation Efficiencies, and Dynamical Evolution of 18,000 $\ion{H}{2}$ Regions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.22789-b31b1b.svg)](https://arxiv.org/abs/2509.22789)<mark>Appeared on: 2025-09-30</mark> -  _Accepted for publication in ApJL; main text: 13 pages, 4 figures, 1 table_

</div>
<div id="authors">

D. Pathak, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present measurements of the masses associated with $\sim18,000$ $\ion{H}{2}$ regions across 19 nearby star-forming galaxies by combining data from JWST, HST, MUSE, ALMA, VLA, and MeerKAT from the multi-wavelength PHANGS survey. We report 10 pc-scale measurements of the mass of young stars, ionized gas, $\edit{and}$ older disk stars coincident with each $\ion{H}{2}$ region, as well as the initial and current mass of molecular gas, atomic gas, and swept-up shell material $\edit{, estimated from lower resolution data}$ . We find that the mass of older stars dominates over young stars at $\gtrsim10\rm pc$ scales, and ionized gas $\edit{exceeds the stellar mass}$ in most optically bright $\ion{H}{2}$ regions. Combining our mass measurements for a statistically large sample of $\ion{H}{2}$ regions, we derive 10 pc scale star-formation efficiencies $\approx6{-}17\%$ for individual $\ion{H}{2}$ regions. Comparing each region's self-gravity with the ambient ISM pressure and total pressure from pre-supernova stellar feedback, we show that most optically bright $\ion{H}{2}$ regions are over-pressured relative to their own self-gravity and the ambient ISM pressure, and that they are hence likely expanding into their surroundings. Larger $\ion{H}{2}$ regions in galaxy centers approach dynamical equilibrium. The self-gravity of regions is expected to dominate over $\edit{pre-supernova stellar feedback pressure at $\gtrsim130\rm pc$ and $60\rm pc$ scales}$ in galaxy disks and centers, respectively, $\edit{but is always sub-dominant to the ambient ISM pressure on \ion{H}{2} region scales}$ . Our measurements have direct implications for the dynamical evolution of star-forming regions and the efficiency of stellar feedback in ionizing and clearing cold gas.

</div>

<div id="div_fig1">

<img src="tmp_2509.22789/./mass_violin_plots.png" alt="Fig1.1" width="50%"/><img src="tmp_2509.22789/./Mnew_MHII_MOld_MMol_vsRcirc.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** The distribution of masses associated with our sample of $\sim18,000$\ion{H}{2} regions (solid violins) and the subsample of $\sim 4,000$ regions where we can calculate [$\ion${S}{2}] densities (dashed violins), including contribution from young clusters ($M_{\rm New}$, in blue, see \S\ref{sec:estimating-MNew}), ionized gas mass ($M_{\rm Ion}$; Strömgren in light green, [$\ion${S}{2}]-emitting clump in dark green, see \S\ref{sec:estimating-MHII}), old stellar mass ($M_{\rm Old}$; red, see \S\ref{sec:estimating-MOld}), initial molecular gas mass ($M_{\rm Mol}^{\rm init}$ in pink, \S\ref{sec:estimating-MMol}), and initial atomic gas mass ($M_{\rm Atom}^{\rm init}$ in gray, \S\ref{sec:other-mass-comps}) as violin plots with the median and $16^{\rm th}{-}84^{\rm th}$ range included. Below we show how $M_{\rm New}$, $M_{\rm Ion}$, $M_{\rm Old}$, and $M_{\rm Mol}^{\rm init}$ scale with physical size ($R_{\rm circ}$). Contours show the $16^{\rm th}{/}25^{\rm th}{/}50^{\rm th}{/}75^{\rm th}{/}84^{\rm th}$ percentiles, and we include $\log-\log$ fits (bold lines, repeated as dashed lines in each panel) to the binned medians (hexagons), and the $16^{\rm th}{-}84^{\rm th}$ percentile scatter in each bin (error bars on hexagons). Attached to each scatter plot are stacked histograms of region masses for galaxy centers (orange) and disks (blue) \citep[following][]{2021QUEREJETA}; the sample median (dashed line), and $16^{\rm th}{-}84^{\rm th}$ percentile range (hatches).
 (*fig:Mass-Terms-All4*)

</div>
<div id="div_fig2">

<img src="tmp_2509.22789/./M_vs_R_LHacorr.png" alt="Fig2" width="100%"/>

**Figure 2. -** Variation in the different mass components relative to the young cluster mass $M_{\rm New}$ for the full sample of $\ion${H}{2} regions, binned in $L_{\rm H\alpha}^{\rm corr}$(top left) and $R_{\rm circ}$(top right), and summarized as histograms (bottom left). $M_{\rm Ion, SII}/M_{\rm New}$(dark green) and $M_{\rm Ion, Str\ddot{o}mgren}/M_{\rm New}$(light green), $M_{\rm Old}/M_{\rm New}$(red), $M_{\rm Mol }^{\rm init}/M_{\rm New}$(dashed pink), and $M_{\rm Atom}^{\rm init}/M_{\rm New}$(dashed gray) are included for comparison. The dashed line indicates where a mass component is equal to $M_{\rm New}$. The bottom right panel includes the distribution of the measured $R_{\rm circ}$(black), the estimated radius at which $M_{\rm New} = M_{\rm Mol}^{\rm init}$(dashed pink), and radius where $M_{\rm New} = M_{\rm Old}$(red).
 (*fig:Mass-v-R-LHa*)

</div>
<div id="div_fig3">

<img src="tmp_2509.22789/./SFE_est.png" alt="Fig3.1" width="33%"/><img src="tmp_2509.22789/./SFE_MMol_supplement2.png" alt="Fig3.2" width="33%"/><img src="tmp_2509.22789/./SFE_MMol_supplement.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** **Top:** Summarizing $\ion${H}{2} region-scale SFEs. Data density contours for the $5^{\rm th}{/}16^{\rm th}{/}25^{\rm th}{/}50^{\rm th}{/}75^{\rm th}{/}84^{\rm th}{/}95^{\rm th}$ percentile SFE using $M_{\rm Ion, SII}$(dark green), $M_{\rm Ion, Str\ddot{o}mgren}$(light green), and $M_{\rm Mol}^{\rm init}$(purple). The full histogram and medians as before (dashed line, printed) for $M_{\rm Ion, Str\ddot{o}mgren}$ and $M_{\rm Mol}^{\rm init}$ are included. We also include the medians using $M_{\rm Mol}^{\rm init}$ at 90 and 60 pc resolution.
\edit{**Middle:** From left to right, the fraction of current gas and stars in young stars, the moderate correlation (Spearman $\rho = 0.42$) and large scatter between $M_{\rm New}$ and $ \langle \Sigma_{\rm Mol} \rangle_{150 \rm   pc}$, and three estimates of $M_{\rm New} / M_{\rm Cloud}$ vs $ \langle \Sigma_{\rm Mol} \rangle_{150 \rm   pc}$}.
**Bottom:** From left to right, the fraction of initial gas seen in stars and ionized gas, fraction of depleted initial molecular gas in stars and ionized gas, statistical local depletion in molecular gas towards $\ion${H}{2} regions at our fiducial 150 pc resolution, and finally the depletion for smaller samples (where available) at 90 pc (purple) and 60 pc (grey) resolution, relative to the dashed black 1:1 line.
 (*fig:SFE-est*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.22789"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\linsimstackns}{\texttt{LinSimStack}}$
$\newcommand{\linsimstack}{\linsimstackns\;}$
$\newcommand{\mum}{{\ensuremath{\mu \mathrm{m}}}}$
$\newcommand{\reviewEditOne}[1]{#1}$</div>



<div id="title">

# Far-infrared lines hidden in archival deep multi-wavelength surveys: Limits on $\ciium$ at $z\sim 0.3-2.9$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.24211-b31b1b.svg)](https://arxiv.org/abs/2509.24211)<mark>Appeared on: 2025-09-30</mark> -  _Submitted to Astronomy & Astrophysics on July 19, 2025_

</div>
<div id="authors">

S. Agrawal, J. Aguirre, <mark>R. Keenan</mark>

</div>
<div id="abstract">

**Abstract:** _Context:_ Singly-ionized carbon $\cii$ is theorized to be the brightest emission line feature in star-forming galaxies, and hence an excellent tracer of the evolution of cosmic star formation. Archival maps from far-infrared and sub-millimeter surveys potentially contain the redshifted $\ciiumns$ , hidden in the much-brighter continuum emission. _Aim:_ We present a search for aggregate $\ciium$ line emission across the predicted peak of star formation history by tomographically stacking a high-completeness galaxy catalog on broadband deep maps of the COSMOS field and constraining residual excess emission after subtracting the continuum spectral energy distribution (SED). _Methods:_ The COSMOS equatorial $2\deg^2$ patch has been mapped by _Spitzer_ , _Herschel_ , and SCUBA2/JCMT. With the high precision UV-O-IR photometry catalog COSMOS2020, we perform unbiased simultaneous stacking of $\sim360,000$ photometric redshifts on these confusion-limited maps to resolve the sub-THz radiation background.By subtracting an continuum SED model with conservative uncertainty estimation and completeness correction through comparison to the _COBE_ /FIRAS monopole spectrum, we obtain tomographic constraints on the sky-averaged $\ciium$ signal within the three SPIRE maps: $11.8 \pm 10.2$ , $11.0 \pm 8.7$ , $9.6 \pm 9.8$ , and $9.2 \pm 6.6$ $k$ Jy/sr at redshifts $z\sim 0.65$ , $\sim1.3$ , $\sim2.1$ , and $\sim2.6$ respectively, corresponding to $1-1.4\sigma$ significance in each bin. _Results:_ Our $3\sigma$ upper limits are in tension with past $z\sim 2.6$ results from cross-correlating SDSS-BOSS quasars with high-frequency _Planck_ maps, and indicate a much less dramatic evolution ( $\sim\times 7.5$ ) of mean $\cii$ intensity across the peak of star formation history than collisional excitation models or frameworks calibrated to the tentative _Planck_ xBOSS measurement. We discuss this tension, particularly in the context of in-development surveys (TIM, EXCLAIM) that will map this $\cii$ at high redshift resolution. _Conclusion:_ Having demonstrated stacking in broadband deep surveys as a complementary methodology to next-generation spectrometers for line intensity mapping, our novel methods can be extended to upcoming galaxy surveys such as _Euclid_ , as well as to place upper limits on fainter atomic and molecular lines.

</div>

<div id="div_fig1">

<img src="tmp_2509.24211/./constraints.png" alt="Fig4" width="100%"/>

**Figure 4. -** $\reviewEditOne${Our measurements of \ciium at $z\sim 0.3-2.9$, plotted with estimates obtained from measurements of the \ciiumns-LF in the local universe and $z\sim5$. Also shown are theoretical predictions from C+ evolution models; our $3\sigma$ upper limits disfavor high-temperature collisional excitation frameworks and best-fit empirical models calibrated to the Pullen_2018 _Planck_ measurement. Our $1\sigma$ results have a higher chance of agreement with SFR-scaling models, which calibrate C+ luminosity to the star formation rate of sources.
Additionally, we plot the _COBE_/FIRAS measurement of the monopole spectrum of the Cosmic Infrared Background as a function of wavelength matched to the rest frame redshift of \ciium emission. We find \ciium likely does not contribute more than a few percent of the total Cosmic Infrared Background, and the intensity history to have undergone a less dramatic evolution across cosmic noon, rising and falling within an order of magnitude.
} (*fig:constraints*)

</div>
<div id="div_fig2">

<img src="tmp_2509.24211/./filter_curves_main.png" alt="Fig1" width="100%"/>

**Figure 1. -**  Overview of data used in our analysis. We include transmission curves (left vertical, bottom horizontal axes) of the eight broadband maps used, from _Spitzer_, _Herschel_, and SCUBA-2. We overplot the rest frame wavelengths of some far-infrared emission lines, including \ciiumns; these lines are redshifted into the broadband maps. Notably, the top horizontal axis labels translate the observer wavelengths into $\reviewEditOne${the} rest frame redshift $z$ of a \ciium emitter. We trace the $z$ distribution of the COSMOS2020 photometric catalog (in bins of $\Delta z = 0.05$), with number counts on the right vertical axis; $\reviewEditOne${number counts for the selection into star-forming or quiescent emitters is also shown}. Redshifted C+ emission will end up in the SPIRE maps for specific $z$ ranges. Finally, at the top, we demarcate the $z$-binning used for stacking with $\linsimstack$; bins at $z\sim0.65$, 1.3, 2.1, \& 2.6 were chosen such that they overlap with the SPIRE bands. (*fig:filter_curves*)

</div>
<div id="div_fig3">

<img src="tmp_2509.24211/./compare.png" alt="Fig6" width="100%"/>

**Figure 6. -** Our constraints, incl. $3\sigma$ upper bounds, plotted with the 95\% C.L. bounds from the Yang_2019(a followup of Pullen_2018) measurement at $z\sim 2.6$. We do not find \ciium at the levels previously tentatively detected; our best estimates are $\sim7.5\times$ lower, with our $3\sigma$ upper bound at $z\sim2.6$ lower than the 95\% C.L. interval.
Also plotted are expected $3\sigma$ sensitivity limits of two in-development surveys  (Pullen_2023,  timconstraints)  that will measure C+ at high $R$. We discuss these and this tension further in text (Section \ref{sec:results}). (*fig:compare*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.24211"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\uproman}[1]{\uppercase\expandafter{\romannumeral#1}}$
$\newcommand{\ur}[1]{\uproman{#1}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\ha}{\ensuremath{{\rm H_{\alpha}}}}$
$\newcommand{\hb}{\ensuremath{{\rm H_{\beta}}}}$
$\newcommand{\der}{{\rm d}}$
$\newcommand{\feh}{{\rm[Fe/H]}}$
$\newcommand{\afe}{{\rm[\rm \alpha/Fe]}}$
$\newcommand{\teff}{\rm T_{eff}}$
$\newcommand{\logg}{\rm \log{g}}$
$\newcommand{\vmic}{\rm v_{mic}}$
$\newcommand{\vbrd}{\rm v_{brd}}$
$\newcommand{\dnlte}{\rm \Delta_{NLTE}}$
$\newcommand{\chem}[2]{\rm ^{#2}#1}$
$\newcommand{\tross}{\rm \tau_{ross}}$
$\newcommand{\ltross}{\rm \log \tau_{ross}}$
$\newcommand{\mA}{{\rm mÅ}}$
$\newcommand{\Elow}{E_{\rm low}}$
$\newcommand{\Eup}{E_{\rm up}}$
$\newcommand{\SH}{S\!_{\rm H}}$
$\newcommand{\opd}{\log \tau_{\rm 5000}}$
$\newcommand{\lin}{ \lambda}$
$\newcommand{\tda}{\langle{\rm 3D}\rangle}$
$\newcommand{\multitd}{MULTI3D}$
$\newcommand{\dis}{\texttt{DISPATCH} }$
$\newcommand{\medis}{\texttt{M3DIS}}$
$\newcommand{\logtr}{\log{\tau_{\rm ross}}}$
$\newcommand{\iso}[2]{\rm ^{#2} #1 }$
$\newcommand{\PEremark}[1]{\textcolor{red}{#1 /PE.}}$
$\newcommand{\MBremark}[1]{\textcolor{red}{#1 /MB.}}$
$\newcommand{\RHremark}[1]{\textcolor{red}{#1 /RH.}}$
$\newcommand{\ANremark}[1]{\textcolor{red}{#1 /{Å}N.}}$
$\newcommand{\BPremark}[1]{\textcolor{red}{#1 /BP.}}$
$\newcommand{\JKremark}[1]{\textcolor{red}{#1 /JK.}}$
$\newcommand{\APremark}[1]{\textcolor{red}{#1 /AP.}}$
$\newcommand{\added}[1]{\ifhighlight\textbf{#1}\else#1\fi}$
$\newcommand{\mb}[1]{\ifhighlight\textbf{\textcolor{magenta}{#1}}\else#1\fi}$
$\newcommand{\revisedPE}[1]{\ifhighlight\textbf{\textcolor{green}{#1}}\else#1\fi}$
$\newcommand{\addedPE}[1]{\ifhighlight\textbf{\textcolor{blue}{#1}}\else#1\fi}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# M3DIS - A grid of 3D radiation-hydrodynamics stellar atmosphere models for stellar surveys.: II. Carbon-enhanced metal-poor stars${\thanks{All \medis 3D RHD CEMP models are available under \url{https://nlte.mpia.de}}}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.24555-b31b1b.svg)](https://arxiv.org/abs/2509.24555)<mark>Appeared on: 2025-09-30</mark> -  _16 pages, 13 figures, accepted by A&A_

</div>
<div id="authors">

<mark>P. Eitner</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>R. Hoppe</mark>, <mark>N. Storm</mark>

</div>
<div id="abstract">

**Abstract:**            Understanding the origin and evolution of carbon-enhanced metal-poor (CEMP) stars is key to tracing the Galaxy's early chemical enrichment. We investigate how realistic 3D radiation-hydrodynamics (RHD) model atmospheres affect carbon abundances in CEMP stars and implications for their classification and Galactic chemical evolution (GCE). We focus on biases from traditional 1D hydrostatic models. We use the M3DIS code to compute 3D RHD model atmospheres for main-sequence and sub-giant stars over a wide range of metallicities and carbon enhancements. Synthetic spectra of the CH G-band are calculated with 3D radiative transfer and compared to spectra from classical 1D MARCS models. We derive abundance corrections and apply them to a large SAGA database sample to quantify effects on the carbon abundance distribution and CEMP classification. Our new 3D CEMP models predict cooler upper atmospheric layers than in 1D models, resulting in stronger CH absorption and lower inferred carbon abundances by up to -0.9 dex at the lowest metallicities. Carbon enhancement in the atmosphere itself increases molecular opacities and leads to radiative re-heating, partly offsetting adiabatic cooling in 3D models and reducing 3D-1D abundance corrections. Applying these corrections lowers the CEMP fraction by up to 20% below [Fe/H]=-3 and alters the relative contribution of CEMP sub-classes. The fraction of CEMP-no stars increases while the number of CEMP-r/s stars decreases, due to the downward revision of absolute carbon abundances. These changes bring the Galactic carbon distribution into better agreement with GCE models assuming a 20% contribution from faint supernovae. Realistic model atmospheres are essential to reliably reconstruct the Galaxy's early chemical enrichment history.         

</div>

<div id="div_fig1">

<img src="tmp_2509.24555/./plots/CEMP_t57.50g45.00m-5.000_v1.0_vs_CEMP_t52.50g30.00m-5.000_v1.0_all.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of the mean temperature stratification between 1D scaled-solar MARCS (red solid lines), 3D scaled-solar (blue solid lines), and 3D CEMP (blue crosses) models against mean density (top) and optical depth at $\rm 500 nm$(bottom). Left: MS model. Right: sub-giant model. For the MS model in the left panels we furthermore include a CEMP MARCS model (provided by B. Plez, priv. conv., shown with red crosses) (*fig:ttau_ttho*)

</div>
<div id="div_fig2">

<img src="tmp_2509.24555/./plots/yoon_beers_reduces_no_reclassification.png" alt="Fig5" width="100%"/>

**Figure 5. -** Yoon-Beers \citep{Yoon2016} diagram, A(C) against [Fe/H] for the 3D-corrected measurements for Galactic stars from the literature (see text). The grey dotted line represents the dividing CEMP classification abundance of $\rm[C/Fe]=0.7$. Coloured lines show the major axis of the ellipse representing the distribution of different CEMP groups. In 1D (dashed lines), ellipses are drawn similar to \cite{Yoon2016}, their Fig. 1. In 3D (solid lines) ellipses are obtained from fitting a Gaussian mixture model to the data. (*fig:yoon-beers*)

</div>
<div id="div_fig3">

<img src="tmp_2509.24555/./plots/CEMP_t57.50g45.00m-5.000_v1.0_structure_3D1D.png" alt="Fig11.1" width="50%"/><img src="tmp_2509.24555/./plots/CEMP_t52.50g30.00m-5.000_v1.0_structure_3D1D.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** Comparison of mean temperature stratification between the \texttt{DISPATCH} 3D CEMP model and the 1D MARCS model on the optical depth scale. Left: $\rm T_{eff} = 5750 K, log(g)=4.5$, Right: $\rm T_{eff} = 5250 K, log(g)=3.0$. Horizontal bars at the bottom correspond to the approximate formation region of the G-band in 1D and 3D. (*fig:average_profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.24555"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}[1]{$
$    \oldbibliography{#1}$
$    \setlength{\itemsep}{0pt}$
$}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

#     Polarization at millimeter wavelengths caused by drifting grains\ in protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.24594-b31b1b.svg)](https://arxiv.org/abs/2509.24594)<mark>Appeared on: 2025-09-30</mark> -  _18 pages, 14 figures, to be published in Astronomy & Astrophysics (A&A)_

</div>
<div id="authors">

M. Lietzow-Sinjen, S. Reissl, <mark>M. Flock</mark>, S. Wolf

</div>
<div id="abstract">

**Abstract:** During the evolution of protoplanetary disks, dust grains start to grow, form larger particles, settle to the midplane, and rearrange the disk, mainly by the inward radial drift.    Because of this, dust pebbles with an irregular shape usually align mechanically and thus cause polarization signatures in their thermal radiation due to dichroic emission or absorption. The goal of this paper is to evaluate the potential to trace the impact of mechanical grain alignment in protoplanetary disks on the observed degree and orientation of linear polarization at millimeter wavelengths. We combined 3D radiation hydrodynamical simulations to determine the density distribution and the velocity field of gas and dust particles, Monte Carlo dust-gas interaction simulations to calculate the mechanical alignment of dust in a gas flow, and, finally, 3D Monte Carlo polarized radiative transfer simulations to obtain synthetic polarimetric observations. We find that large grains, which contribute the most to the net polarization, are potentially mechanically aligned in the protoplanetary disk under the effect of the vertical shear instability (VSI).    Thereby, the drift velocity is parallel to the rotational disk axis.    Assuming oblate dust grains that are aligned with their short axis parallel to the direction of the drift velocity, the resulting polarization is usually along the major axis of the disk.    This is in contrast to typical drift models that propose either a radial or azimuthal drift velocity component. If hydrodynamical instabilities, such as the VSI, dominate the kinematics in protoplanetary disks, the mechanical alignment of dust is a promising mechanism for grain alignment in these systems.    In that case, the resulting millimeter polarization allows us to trace the orientation of aligned millimeter-sized grains.

</div>

<div id="div_fig1">

<img src="" alt="Fig11.1" width="25%"/><img src="" alt="Fig11.2" width="25%"/><img src="" alt="Fig11.3" width="25%"/><img src="" alt="Fig11.4" width="25%"/>

**Figure 11. -** 
        Surface brightness map of the total emitted thermal radiation at a wavelength of \SI{1.3}{mm}(left) and \SI{10}{mm}(right) overlaid with polarization vectors at these wavelengths.
        In the top row (a, b), the polarization arises from thermal emission and dichroic extinction of aligned dust grains only.
        At wavelengths of \SI{1.3}{mm} and \SI{10}{mm}, only \SI{100}{\um} and \SI{1}{mm}-sized grains are assumed to be aligned, respectively.
        In contrast, in the bottom row (c, d), the polarization arises from self-scattered radiation only.
        The disk is inclined by \ang{45}.
        See Sects. \ref{subsec:polarization_alignment} and \ref{subsec:polarization_scattering} for details.
     (*fig:sb_align_selfsca_i45*)

</div>
<div id="div_fig2">

<img src="" alt="Fig5.1" width="33%"/><img src="" alt="Fig5.2" width="33%"/><img src="" alt="Fig5.3" width="33%"/>

**Figure 5. -** 
        Wavelength dependence of the radial profile of the polarization degree.
        In the top and middle figure, only \SI{100}{\um} and \SI{1}{mm}-sized grains are aligned and contribute to the polarization, respectively.
        In the bottom figure, the polarization arises from self-scattered radiation only.
        The disk is inclined by \ang{45}.
        See Sects. \ref{subsec:polarization_alignment} and \ref{subsec:polarization_scattering} for details.
     (*fig:pol_rad_i45*)

</div>
<div id="div_fig3">

<img src="" alt="Fig7.1" width="50%"/><img src="" alt="Fig7.2" width="50%"/>

**Figure 7. -** 
        Surface brightness map of the total emitted thermal radiation at a wavelength of \SI{3}{mm} overlaid with corresponding polarization vectors.
        The polarization arises from thermal emission and dichroic absorption of aligned \SI{1}{mm}-sized dust grains (top) and from self-scattered radiation (bottom) only.
        The disk is inclined by \ang{45}.
        See Sect. \ref{subsec:flip_polarization} for details.
     (*fig:sb_a1mm_selfsca_i45_w3mm*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.24594"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

127  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
